# Multivariate Analsysis

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import statsmodels.api as sm
import scipy.stats as stats
import matplotlib.pyplot as plt

In [ ]:
%matplotlib widget
%matplotlib inline
%matplotlib ipympl
import warnings

warnings.filterwarnings('ignore')


In [ ]:
dataset = pd.read_csv(r"C:\work\water_use\ml_experiments\annual_v_0_0\clean_train_db.csv")
raw_dataset = pd.read_csv(r"C:\work\water_use\mldataset\ml\training\train_datasets\Annual\raw_wu_annual_training.csv")
pop_info = pd.read_csv(r"pop_info.csv")

In [ ]:
# transfere pop from pop_info to dataset
pop_info['pop'] = pop_info['pop_swud16'].copy()
mask = (pop_info['pop'].isna()) | (pop_info['pop']==0)
pop_info.loc[mask, 'pop'] = pop_info[mask]['plc_pop_interpolated']
mask = (pop_info['pop'].isna()) | (pop_info['pop']==0)
pop_info.loc[mask, 'pop'] = pop_info[mask]['TPOPSRV']
mask = (pop_info['pop'].isna()) | (pop_info['pop']==0)
pop_info.loc[mask, 'pop'] = pop_info[mask]['tract_pop']
dataset = dataset[dataset['Ecode_num']==0]

pop_df = pop_info[['sys_id', 'pop', 'Year']]
dataset = dataset.merge(pop_df, right_on=['sys_id', 'Year'], left_on=['sys_id', 'Year'] , how = 'left')

In [ ]:
# transfer processed income and house ag from raw to master dataset
cols_to_transfer = ['Year', 'sys_id']
for col in raw_dataset.columns:
    if  ("h_age" in col) \
    or ("income" in col) \
    or ("median_h_year" in col) \
    or ("av_house_age" in col) \
    or ("n_houses") in col:        
        cols_to_transfer.append(col)
        if col in dataset.columns:
            del(dataset[col])
    
raw_dataset = raw_dataset[cols_to_transfer]       


In [ ]:
dataset = dataset.merge(raw_dataset, right_on=['sys_id', 'Year'], left_on=['sys_id', 'Year'] , how = 'left')

Remove extreme population density values

In [ ]:
# maximum pop density in the us is 22K/km2
# I assume the minimum reasonable density of people to be 22 or 3% percintile)
# https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population_density
plt.figure()
np.log10(dataset['pop']/dataset['WSA_SQKM']).hist(bins = 100)
plt.show()
dataset['pop_density'] = dataset['pop']/dataset['WSA_SQKM']
len_before = len(dataset)
dataset = dataset[dataset['pop_density']<= 22000]
dataset = dataset[dataset['pop_density']>= 22]
ratio = len(dataset)/len_before
print("Percentage of rows with extrem pop density {}".format(100*(1-ratio)))

Remove extreme per capita

In [ ]:

max_allowed_pc = 500
min_allowed_pc = 20
min_allowed_pop = 100


train_db = dataset[dataset['wu_rate']>0]
len_before = len(train_db)
train_db['pc'] = train_db['wu_rate']/train_db['pop']
train_db = train_db[train_db['pop']>min_allowed_pop]
mask = (train_db['pc']>=min_allowed_pc) & (train_db['pc']<=max_allowed_pc)
train_db = train_db[mask]
ratio = 100 * (1-(len(train_db)/len_before))
print("Percentage of extreme PC and Population < 100 is {}".format(ratio))

## Effect of income on per-capita use

In [ ]:
def multi_hyp_tests(var1 = 'average_income' , var2 = 'pc', step = 0.05):
    """ We use income as a variable, but other variables can be used"""
    # Partition the data  
    quantiles = np.arange(0,1,step)
    pc_sets = {}
    pc_income = {}
    ttest_results = []
    for q in quantiles:
        income0 = train_db[var1].quantile(q)
        income1 = train_db[var1].quantile(q+step)
        mask1 = train_db[var1]>= income0
        mask2 = train_db[var1]<income1
        key = q + step/2.0
        pc_sets[key] =  train_db.loc[mask1 & mask2, var2].values
        pc_income[key] = (income0 + income1)/2.0
    set_keys = sorted(list(pc_sets.keys()))
    hyp_test_results = []
    for s1 in set_keys:
        for s2 in set_keys:
            if s2>s1:
                ht = stats.ttest_ind(a=pc_sets[s1], b=pc_sets[s2], equal_var=False)
                curr = [s1, s2, pc_income[s1], pc_income[s2],  ht.statistic, ht.pvalue]
                hyp_test_results.append(curr)
    columns = ['q1', 'q2', 'income1', 'income2', 'tstat', 'pvalue']
    hyp_test_results = pd.DataFrame(hyp_test_results, columns=columns)    
    del(pc_sets)
    return hyp_test_results


In [ ]:
hyp_test_results = multi_hyp_tests(var1 = 'average_income' , var2 = 'pc', step = 0.05)
plt.figure()
plt.scatter(hyp_test_results['q1'], hyp_test_results['q2'], c = np.log10(hyp_test_results['pvalue']), cmap = 'jet')
plt.colorbar()
reject = hyp_test_results[hyp_test_results['pvalue']>=0.05]
plt.scatter(reject['q1'], reject['q2'], facecolors='none', s = 100, edgecolors='k')

# plt.gca().set_yscale('log')
# plt.gca().set_xscale('log')


In [ ]:
# plt.figure()
# #plt.scatter(train_db['average_income'], train_db['pc'], s=1, c = [0.7,0.7,0.7])
# plt.plot(ttest_results['Income'], ttest_results['ttest_stat'])
# secax = plt.gca().twinx()
# secax.plot(ttest_results['Income'], np.log10(ttest_results['ttest_pvalue']), 'k' )

# plt.figure()
# #plt.plot(ttest_results['Income'], ttest_results['u_median'])
# arr = stats.binned_statistic(train_db['average_income'],  train_db['pc'], statistic='median', bins=20)
# #plt.scatter(train_db['average_income'], train_db['pc'])
# plt.plot(arr.bin_edges[1:], arr.statistic)
# #arr = stats.binned_statistic(train_db['average_income'],  train_db['pc'], statistic='mean', bins=10)

# #plt.plot(arr.bin_edges[1:], arr.statistic)


In [ ]:
slopes = []
sys_db = train_db.groupby(by = 'sys_id').mean()
for i in range(4000):
    xy = train_db[['average_income', 'pc']]
    n = np.random.randint(300)
    a = xy.sample(n, random_state = np.random.randint(1000))
    b = xy.sample(n, random_state = np.random.randint(1000))
    sign = ((a['pc'].values-b['pc'].values)/(a['average_income'].values-b['average_income'].values))>0
    ss = 100*np.sum(sign)/n
    slopes.append(ss)


In [ ]:
plt.figure()
xy = plt.hist(slopes, bins = 50)
plt.plot([50, 50], [0, np.max(xy[0])], label = "50% Cutoff")
plt.xlabel("Fraction of Positive slopes ($\delta PerCapita/\delta Income$)")
plt.ylabel("Frequency")

### Use non-parametric tests
* Spearman's rank correlation coefficient

In [ ]:
from scipy.stats import spearmanr
xy = train_db[train_db['average_income']<train_db['average_income'].quantile(0.99)]
arr = stats.binned_statistic(xy['average_income'],  xy['pc'], statistic='mean', bins=20)
#coef, p = spearmanr(train_db['average_income'], train_db['pc'])
coef, p = spearmanr(arr.bin_edges[1:], arr.statistic)
print('Spearmans correlation coefficient: %.3f' % coef)
# interpret the significance
alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

* Kendall rank correlation coefficient

In [ ]:
from scipy.stats import kendalltau
# calculate kendall's correlation
xy = train_db[train_db['average_income']<train_db['average_income'].quantile(0.99)]
arr = stats.binned_statistic(xy['average_income'],  xy['pc'], statistic='mean', bins=100)
#coef, p = kendalltau(xy['average_income'], xy['pc'])
coef, p = kendalltau(arr.bin_edges[1:], arr.statistic)
print('Kendall correlation coefficient: %.3f' % coef)
# interpret the significance
alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

### Scatter Plot

In [ ]:
def plot_scatter(varX = 'average_income', varY = 'pc', xlimits = [30000,120000], 
                ylimits = [20,300], xlabel = "Average Income in Service Area",
                ylabel = "Gallon Per Capita per day Water Use", xlog = False,
                ylog = False):
    left, width = 0.1, 0.65
    bottom, height = 0.1, 0.65
    spacing = 0.005

    x = train_db[varX]
    y = train_db[varY]
    arr = stats.binned_statistic(x,  y, statistic='mean', bins=50)
    pc_limits = np.array(ylimits)
    income_limits = np.array(xlimits)
    rect_scatter = [left, bottom, width, height]
    rect_histx = [left, bottom + height + spacing, width, 0.2]
    rect_histy = [left + width + spacing, bottom, 0.2, height]

    # start with a rectangular Figure
    plt.figure(figsize=(8, 8))

    ax_scatter = plt.axes(rect_scatter)
    ax_scatter.tick_params(direction='in', top=True, right=True)
    ax_histx = plt.axes(rect_histx)
    ax_histx.tick_params(direction='in', labelbottom=False)
    ax_histy = plt.axes(rect_histy)
    ax_histy.tick_params(direction='in', labelleft=False)

    # the scatter plot:
    ax_scatter.scatter(x, y, s = 1, c = [0.7,0.7,0.7])
    ax_scatter.plot(arr.bin_edges[1:], arr.statistic)
    ax_scatter.set_ylim((pc_limits[0], pc_limits[1]))
    ax_scatter.set_xlim((income_limits[0], income_limits[1]))
    ax_scatter.set_xlabel(xlabel)
    ax_scatter.set_ylabel(ylabel)
    

    # bins = np.arange(-lim, lim + binwidth, binwidth)
    ax_histx.hist(x, bins=100)
    ax_histy.hist(y, bins=100, orientation='horizontal')

    ax_histx.set_xlim(ax_scatter.get_xlim())
    ax_histy.set_ylim(ax_scatter.get_ylim())
    
    if xlog:
        ax_scatter.set_xscale('log')
        ax_histx.set_xscale('log')
    if ylog:
        ax_scatter.set_yscale('log')
        ax_histx.set_yscale('log')

    plt.show()

In [ ]:
plot_scatter(varX = 'average_income', varY = 'pc', xlimits = [30000,120000], 
                ylimits = [20,300], xlabel = "Average Income in Service Area",
                ylabel = "Gallon Per Capita per day Water Use")

In [ ]:
# lot size + income?
# cost of living + income?
# median p25
# https://www.aceee.org/research-report/u2011

## How population density affects per-capita-water use

In [ ]:
## Effect of population density
# 1) split the data based on a certain income into two sets
# 2) test if the mean/median of the two population are significantly different
hyp_test_results = multi_hyp_tests(var1 = 'pop_density' , var2 = 'pc', step = 0.05)
plt.figure()
plt.scatter(hyp_test_results['q1'], hyp_test_results['q2'], c = np.log10(hyp_test_results['pvalue']), cmap = 'jet')
plt.colorbar()
reject = hyp_test_results[hyp_test_results['pvalue']>=0.05]
plt.scatter(reject['q1'], reject['q2'], facecolors='none', s = 100, edgecolors='k')

In [ ]:
train_db['log_pop_density'] = np.log10(train_db['pop_density'])
plot_scatter(varX = 'log_pop_density', varY = 'pc', xlimits = [1,4], 
                ylimits = [20,300], xlabel = "Log10 of Density",
                ylabel = "Gallon Per Capita per day Water Use")
del(train_db['log_pop_density'])

In [ ]:
xy = train_db[['pop_density', 'pc']]
n = 100
slopes = []
for i in range(4000):
    n = np.random.randint(300)
    a = xy.sample(n, random_state = np.random.randint(1000))
    b = xy.sample(n, random_state = np.random.randint(1000))
    sign = ((a['pc'].values-b['pc'].values)/(a['pop_density'].values-b['pop_density'].values))>0
    ss = np.sum(sign)/n
    slopes.append(ss)
plt.figure()
xy = plt.hist(slopes, bins = 100)
plt.plot([0.5, 0.5], [0, np.max(xy[0])])

In [ ]:
def Spearman_corr(feature = 'pop_density', target = 'pc', feature_trim = 0.99, bins = 20):
    from scipy.stats import spearmanr
    
    xy = train_db[train_db[feature]<train_db[feature].quantile(feature_trim)]
    xy = xy.dropna(axis=0)
    arr = stats.binned_statistic(xy[feature],  xy[target], statistic='mean', bins=bins)
    xval = arr.bin_edges[1:]
    yval = arr.statistic
    mask = np.logical_not(np.isnan(yval))
    xval= xval[mask]
    yval = yval[mask]
    #coef, p = spearmanr(train_db['average_income'], train_db['pc'])
    coef, p = spearmanr(xval, yval)
    print('Spearmans correlation coefficient: %.3f' % coef)
    # interpret the significance
    alpha = 0.05
    if p > alpha:
        print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
    else:
        print('Samples are correlated (reject H0) p=%.3f' % p)
    return coef, p

def kendalltau_corr(feature = 'pop_density', target = 'pc', feature_trim = 0.99, bins = 20):
    from scipy.stats import kendalltau   
    xy = train_db[train_db[feature]<train_db[feature].quantile(feature_trim)]
    xy = xy.dropna(axis=0)
    arr = stats.binned_statistic(xy[feature],  xy[target], statistic='mean', bins=bins)
    xval = arr.bin_edges[1:]
    yval = arr.statistic
    mask = np.logical_not(np.isnan(yval))
    xval= xval[mask]
    yval = yval[mask]
    #coef, p = spearmanr(train_db['average_income'], train_db['pc'])
    coef, p = kendalltau(xval, yval)
    print('Kendall correlation coefficient: %.3f' % coef)
    # interpret the significance
    alpha = 0.05
    if p > alpha:
        print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
    else:
        print('Samples are correlated (reject H0) p=%.3f' % p)
    return coef, p

In [ ]:
Spearman_corr(feature = 'pop_density', target = 'pc', feature_trim = 0.99, bins = 50)

In [ ]:
kendalltau_corr(feature = 'pop_density', target = 'pc', feature_trim = 0.99, bins = 20)

In [ ]:
skip_list = ['population', 'households2', 'sys_id', 'wu_rate', 'HUC2', 'county_id', 'Ecode_num', 'pop_house_ratio', 'state_id',
            'KG_climate_zone', 'tot_h_age', 'pc']

In [ ]:
all_rank_tests = []
for col in train_db.columns:
    if col in skip_list:
        continue
    coef0, p0 = Spearman_corr(feature = col, target = 'pc', feature_trim = 0.99, bins = 50)
    coef1, p1 = kendalltau_corr(feature = col, target = 'pc', feature_trim = 0.99, bins = 50)
    all_rank_tests.append([col, coef0, p0, coef1, p1])
    
cols = ['Feature', 'Spearman_corr', 'Spear_pval', 'kendall_corr', 'kendall_pval']
all_rank_tests = pd.DataFrame(all_rank_tests, columns=cols);
all_rank_tests

In [ ]:
all_rank_tests.iloc[50:120]

In [ ]:
all_rank_tests['abs'] = np.abs(all_rank_tests['Spearman_corr'])

(income-ztranform/lot_size) or (income * lot_size)


In [ ]:
cc = all_rank_tests.sort_values(by = ['abs'], ascending = False)
cc.iloc[0:50]

In [ ]:
cc.to_csv(r"stats.csv")

In [ ]:
#fig.ax_joint.plot()
arr = stats.binned_statistic(train_db['av_house_age'],  train_db['pc'], statistic='mean', bins=50)
fig.ax_joint.plot(arr.bin_edges[1:], arr.statistic, color = 'r')

In [ ]:
xy = train_db[['av_house_age', 'pc']]
n = 100
slopes = []
for i in range(4000):
    n = np.random.randint(300)
    a = xy.sample(n, random_state = np.random.randint(1000))
    b = xy.sample(n, random_state = np.random.randint(1000))
    sign = ((a['pc'].values-b['pc'].values)/(a['av_house_age'].values-b['av_house_age'].values))>0
    ss = np.sum(sign)/n
    slopes.append(ss)
plt.figure()
xy = plt.hist(slopes, bins = 100)
plt.plot([0.5, 0.5], [0, np.max(xy[0])])
slopes = np.array(slopes)
slopes = slopes[~np.isnan(slopes)]
one_sample = stats.ttest_1samp(slopes, 0.5)

print ("The t-statistic is %.3f and the p-value is %.3f." % one_sample)


In [ ]:
from scipy.stats import spearmanr
xy = train_db[train_db['av_house_age']<train_db['av_house_age'].quantile(0.99)]
arr = stats.binned_statistic(xy['av_house_age'],  xy['pc'], statistic='mean', bins=20)
#coef, p = spearmanr(train_db['average_income'], train_db['pc'])
coef, p = spearmanr(arr.bin_edges[1:], arr.statistic)
print('Spearmans correlation coefficient: %.3f' % coef)
# interpret the significance
alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

In [ ]:
one_sample

In [ ]:
xy = train_db[['pop', 'pc']]
n = 100
slopes = []
for i in range(4000):
    n = np.random.randint(100)
    a = xy.sample(n, random_state = np.random.randint(1000))
    xy = xy.sample(frac=1).reset_index(drop=True)
    b = xy.sample(n, random_state = np.random.randint(1000))
    sign = ((a['pc'].values-b['pc'].values)/(a['pop'].values-b['pop'].values))>=0
    ss = np.sum(sign)/n
    slopes.append(ss)
plt.figure()
xy = plt.hist(slopes, bins = 100)
plt.plot([0.5, 0.5], [0, np.max(xy[0])])
slopes = np.array(slopes)
slopes = slopes[~np.isnan(slopes)]
one_sample = stats.ttest_1samp(slopes, 0.5)

print ("The t-statistic is %.3f and the p-value is %.3f." % one_sample)


In [ ]:
from scipy.stats import kendalltau
# calculate kendall's correlation
xy = train_db[train_db['pop']<train_db['pop'].quantile(0.99)]
arr = stats.binned_statistic(xy['pop'],  xy['pc'], statistic='mean', bins=100)
#coef, p = kendalltau(xy['average_income'], xy['pc'])
coef, p = kendalltau(arr.bin_edges[1:], arr.statistic)
print('Kendall correlation coefficient: %.3f' % coef)
# interpret the significance
alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

In [ ]:
from scipy.stats import spearmanr
xy = train_db[train_db['pop']<train_db['pop'].quantile(0.99)]
arr = stats.binned_statistic(xy['pop'],  xy['pc'], statistic='mean', bins=20)
#coef, p = spearmanr(train_db['average_income'], train_db['pc'])
coef, p = spearmanr(arr.bin_edges[1:], arr.statistic)
print('Spearmans correlation coefficient: %.3f' % coef)
# interpret the significance
alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

In [ ]:

fig= sns.jointplot(np.log10(train_db['pop']), y=train_db['pc'], kind="hist")
arr = stats.binned_statistic(np.log10(train_db['pop']),  train_db['pc'], statistic='mean', bins=50)
fig.ax_joint.plot(arr.bin_edges[1:], arr.statistic, color = 'r')

In [ ]:
x = np.random.rand(10000)
y = np.random.rand(10000)
xy = pd.DataFrame({'x':x, 'y':y})
n = 100
slopes = []
for i in range(4000):
    n = np.random.randint(2000)
    a = xy.sample(n, random_state = np.random.randint(1000))
    b = xy.sample(n, random_state = np.random.randint(1000))
    sign = ((a['x'].values-b['x'].values)/(a['y'].values-b['y'].values))>0
    ss = np.sum(sign)/n
    slopes.append(ss)
plt.figure()
xy = plt.hist(slopes, bins = 100)
plt.plot([0.5, 0.5], [0, np.max(xy[0])])
slopes = np.array(slopes)
slopes = slopes[~np.isnan(slopes)]
one_sample = stats.ttest_1samp(slopes, 0.5)

print ("The t-statistic is %.3f and the p-value is %.3f." % one_sample)
